# Generate ADF4351 registers related funcitons
- https://www.analog.com/media/en/technical-documentation/data-sheets/ADF4351.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = [' ']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],          
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from signal_generators.adf435x.registers_map import _get_registers_map

Populating the interactive namespace from numpy and matplotlib


In [5]:
map = _get_registers_map()

In [21]:
import textwrap

def gen_set_fun_n_property(registers_map):

    def check_char(string):
        string = string.replace('-', '_')
        return string

    def gen_comment(string, width = 90):
        lines = textwrap.wrap( string,   width=width)
        lines = ['\t# {}'.format(l) for l in lines]
        return '\n'.join(lines)

    def gen_set_fun(ele):
        fun_str = "def _set_{}(self, value  = True):\n".format(check_char(ele.name.lower()))
        fun_str += gen_comment(ele.description)
        fun_str += '\n\n'
        fun_str += "\tself._adf._write_element_by_name('{}', int(bool(value)))\n".format(ele.name)
        return fun_str

    def gen_property(ele):    
        fun_str = "@property\n"
        fun_str += "def {}(self):\n".format(check_char(ele.name.lower())) 
        # fun_str += gen_comment(ele.description)
        # fun_str += '\n\n'
        fun_str += "\treturn self._adf.map.elements['{}']['element'].value\n".format(ele.name)
        return fun_str

    i = 0
    
    for reg in map._registers:

        for ele in reg._elements:

            if ele.name != 'index':
                print (gen_set_fun(ele))
                print (gen_property(ele)) 
                
                i += 1
    print(i)

In [22]:
gen_set_fun_n_property(map)

def _set_int(self, value  = True):
	# The 16 INT bits (Bits[DB30:DB15]) set the INT value, which determines the integer part of
	# the feedback division factor. The INT value is used in Equation 1 (see the INT, FRAC, MOD,
	# and R Counter Relationship section). Integer values from 23 to 65,535 are allowed for the
	# 4/5 prescaler; for the 8/9 prescaler, the minimum integer value is 75.

	self._adf._write_element_by_name('INT', int(bool(value)))

@property
def int(self):
	return self._adf.map.elements['INT']['element'].value

def _set_frac(self, value  = True):
	# The 12 FRAC bits (Bits[DB14:DB3]) set the numerator of the fraction that is input to the
	# Σ-Δ modulator. This fraction, along with the INT value, specifies the new frequency
	# channel that the synthesizer locks to, as shown in the RF Synthesizer—A Worked Example
	# section. FRAC values from 0 to (MOD ? 1) cover channels over a frequency range equal to
	# the PFD refer- ence frequency.

	self._adf._write_element_by_name('FRA